In [1]:
!pip install transformers sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=05efd842c82c7b6b535a93b7c565732bc588edc4e3e26a9c0f6d1e9d4e0f51ea
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


# Imports

In [59]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/fdic_paragraphs_improved.csv")
vectors = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/stored_vectors_roberta_improved.csv")

In [60]:
df.head()

,SECTNO,SUBJECT,PARAGRAPH,TEXT
0,§ 302.1,Purpose.,1,The FDIC issues regulations and guidance as pa...
1,§ 302.2,Implementation of the Statement Clarifying the...,1,The Statement describes the official policy of...
2,§ 302.3,Rule of construction.,1,This subpart does not alter the legal status o...
3,§ 303.0,Scope.,1,(a) This part describes the procedures to be f...
4,§ 303.1,Scope.,1,Subpart A prescribes the general procedures fo...


In [61]:
vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.406242,0.489578,0.463368,1.611462,-0.538885,0.123013,0.610867,0.324202,0.416381,-0.095100,...,-0.664674,-0.488142,-0.209462,0.231349,1.147633,0.197752,0.910094,-0.111481,-0.210906,0.045083
1,-0.091986,0.354951,0.249938,1.539177,0.196127,0.266525,0.770281,-0.101251,0.599707,-0.433163,...,0.019105,-0.703642,-0.057147,0.631384,0.755192,0.281061,1.079101,0.650189,-0.230008,0.147003
2,-0.325221,-0.190968,1.133391,0.622859,-0.307541,0.349843,0.354220,0.501340,0.301328,0.331663,...,-0.212974,-0.243739,-0.293114,0.533973,-0.174138,0.318965,1.212714,1.584661,-0.030106,0.536328
3,-0.094506,0.278077,0.959316,0.440302,-0.950825,-0.498754,-0.338766,1.009844,1.629327,-0.643326,...,0.538941,0.157767,0.632817,-0.620262,0.104008,-0.070820,0.569091,-0.448831,-0.233519,-0.503860
4,-0.287259,-0.076270,0.817844,0.436261,-0.953012,-0.294108,-0.322806,0.737543,1.406341,-0.279933,...,0.150143,0.644104,0.146500,-0.194633,0.502899,-0.095035,0.411071,-0.572094,0.061644,-0.515969


In [62]:
model_roberta = SentenceTransformer('/content/drive/MyDrive/Colab Notebooks/models/finetuned_roberta')
#model_mpnet = SentenceTransformer('/content/drive/MyDrive/Colab Notebooks/models/finetuned_mpnet')

In [63]:
question = "What is administrative law judge"

In [88]:
def ask_question(question, vector_df, text_df, model):
  q_vec = model.encode([question])
  cosine_mat = cosine_similarity(q_vec, vector_df)
  top_10_index = np.argsort(cosine_mat[0])[::-1][:10]

  for i, v in enumerate(top_10_index):
    record = text_df.iloc[v]
    print(f"""{i+1}. 
    Text: {record.TEXT}
    Section: {record.SECTNO}
    Subject: {record.SUBJECT}
    Similarity Score: {cosine_mat[0][v]}""")

  

In [89]:
ask_question("what should I include in an application for brokered deposits", vectors, df, model_roberta)

1. 
    Text: (b) Solicitation and acceptance of brokered deposits by insured depository institutions. (1) A well capitalized insured depository institution may solicit and accept, renew or roll over any brokered deposit without restriction by this section.
    Section: § 337.6
    Subject: Brokered deposits.
    Similarity Score: 0.6343690382425367
2. 
    Text: (3) Content of filing. The application shall contain the following: (i) The time period for which the waiver is requested; (ii) A statement of the policy governing the use of brokered deposits in the institution's overall funding and liquidity management program; (iii) The volume, rates and maturities of the brokered deposits held currently and anticipated during the waiver period sought, including any internal limits placed on the terms, solicitation and use of brokered deposits; (iv) How brokered deposits are costed and compared to other funding alternatives and how they are used in the institution's lending and investment a

In [95]:
ask_question("when can exempt records be disclosed", vectors, df, model_roberta)

1. 
    Text: (iv) With respect to any disclosure that is authorized under this paragraph (b)(7), the Director of the Corporation's Division having primary authority over the exempt records, or designee, shall only permit disclosure of records upon determining that good cause exists. If the exempt record contains information derived from depository institution customer financial records, disclosure is to be authorized only upon the condition that the requesting party and the party releasing the records comply with any applicable provision of the RFPA. Before authorizing the disclosure, the Director (or designee) may require that both the party having custody of a copy of a Corporation exempt record and the party seeking access to the record agree to such limitations as the Director (or designee) deems necessary to protect the confidential nature of the record, the financial integrity of any depository institution to which the record relates and the legitimate privacy interests of any p

In [96]:
ask_question("what is the process for requesting disclosure of exempt information", vectors, df, model_roberta)

1. 
    Text: (b) Disclosure authorized. Exempt records or information of the Corporation may be disclosed only in accordance with the conditions and requirements set forth in this paragraph (b). Requests for discretionary disclosure of exempt records or information pursuant to this paragraph (b) may be submitted directly to the Division having primary authority over the exempt records or information or to the FOIA/PA Group for forwarding to the appropriate Division having primary authority over the records sought. Such administrative request must clearly state that it seeks discretionary disclosure of exempt records, clearly identify the records sought, provide sufficient information for the Corporation to evaluate whether there is good cause for disclosure, and meet all other conditions set forth in paragraph (b)(1) through (10) of this section. Information regarding the appropriate FDIC Division having primary authority over a particular record or records may be obtained from the FO

In [92]:
ask_question("what types of information are exempt", vectors, df, model_roberta)

1. 
    Text: (a) Disclosure prohibited. Except as provided in paragraph (b) of this section or by 12 CFR part 310, 2 no person shall disclose or permit the disclosure of any exempt records, or information contained therein, to any persons other than those officers, directors, employees, or agents of the Corporation who have a need for such records in the performance of their official duties. In any instance in which any person has possession, custody or control of FDIC exempt records or information contained therein, all copies of such records shall remain the property of the Corporation and under no circumstances shall any person, entity or agency disclose or make public in any manner the exempt records or information without written authorization from the Director of the Corporation's Division having primary authority over the records or information as provided in this section.
    Section: § 309.6
    Subject: Disclosure of exempt records.
    Similarity Score: 0.7314426711058295
2

In [79]:
ask_question("when does our organization have to publish stress test results", vectors, df, model_roberta)

1. 
    Text: (2) If the Board of Governors of the Federal Reserve System publishes the supervisory stress test results of the covered bank's parent holding company prior to June 15, then such covered bank may publish its stress test results prior to June 15, but no later than July 15, through actual publication by the covered bank or through publication by the parent holding company under paragraph (b) of this section.
    Section: § 325.7
    Subject: Publication of stress test results.
    Similarity Score: 0.8356076629806017
2. 
    Text: (a) Publication date. A covered bank must publish a summary of the results of its stress tests in the period starting June 15 and ending July 15 of the reporting year, provided: (1) Unless the Corporation determines otherwise, if the covered bank is a consolidated subsidiary of a bank holding company or savings and loan holding company subject to supervisory stress tests conducted by the Board of Governors of the Federal Reserve System under 12 CF

In [81]:
ask_question("what are the penalties if an individual makes up a fraud claim to the FDIC", vectors, df, model_roberta)

1. 
    Text: (a) Claims. (1) A person who makes a false, fictitious, or fraudulent claim to the FDIC is subject to a civil penalty of up to $5,000 per claim. A claim is false, fictitious, or fraudulent if the person making the claim knows, or has reason to know, that: (i) The claim is false, fictitious, or fraudulent; or (ii) The claim includes, or is supported by, a written statement that asserts a material fact which is false, fictitious or fraudulent; or (iii) The claim includes, or is supported by, a written statement that: (A) Omits a material fact; and (B) Is false, fictitious, or fraudulent as a result of that omission; and (C) Is a statement in which the person making the statement has a duty to include the material fact; or (iv) The claim seeks payment for providing property or services that the person has not provided as claimed.
    Section: § 308.502
    Subject: Basis for civil penalties and assessments.
    Similarity Score: 0.708759204495192
2. 
    Text: (5) If the FDI

In [75]:
df.iloc[2529].TEXT

'(2) Collateral haircut approach —(i) EAD equation. An FDIC-supervised institution may determine EAD for an eligible margin loan, repo-style transaction, or netting set by setting EAD equal to max {0, [(ΣE−ΣC) + Σ(E s × H s ) + Σ(E fx × H fx )]}, where: (A) ΣE equals the value of the exposure (the sum of the current fair values of all instruments, gold, and cash the FDIC-supervised institution has lent, sold subject to repurchase, or posted as collateral to the counterparty under the transaction (or netting set)); (B) ΣC equals the value of the collateral (the sum of the current fair values of all instruments, gold, and cash the FDIC-supervised institution has borrowed, purchased subject to resale, or taken as collateral from the counterparty under the transaction (or netting set)); (C) E s equals the absolute value of the net position in a given instrument or in gold (where the net position in a given instrument or in gold equals the sum of the current fair values of the instrument or